In [13]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy
import numpy as np
import scipy


In [16]:
data = pd.read_csv("./data.csv")
ports = data["Source Port"].unique()
ports = ports[ports !=1883]
ports = ports[ports != 34366]
ports
data


,No.,Time,Source,Destination,Protocol,Length,Source Port,Message Identifier,Destination Port,Info
0,169,16.982250,192.168.15.6,45.179.91.66,MQTT,68,34366,NaN,1883,Ping Request
1,170,16.998925,45.179.91.66,192.168.15.6,MQTT,68,1883,NaN,34366,Ping Response
2,290,27.448019,192.168.15.6,45.179.91.66,MQTT,89,52269,NaN,1883,Connect Command
3,292,27.464281,192.168.15.6,45.179.91.66,MQTT,354,52269,"1,2",1883,"Subscribe Request (id=1) [devices-1], Publish ..."
4,293,27.464220,45.179.91.66,192.168.15.6,MQTT,70,1883,NaN,52269,Connect Ack
...,...,...,...,...,...,...,...,...,...,...
745,16715,2194.138607,45.179.91.66,192.168.15.6,MQTT,68,1883,NaN,52269,Ping Response
746,17279,2237.073993,192.168.15.6,45.179.91.66,MQTT,68,34366,NaN,1883,Ping Request
747,17280,2237.094071,45.179.91.66,192.168.15.6,MQTT,68,1883,NaN,34366,Ping Response
748,17430,2254.197672,192.168.15.6,45.179.91.66,MQTT,68,52269,NaN,1883,Ping Request


In [17]:
devices = {}
#  : devices será um dicionario que irá ter uma chave referente ao device e o valor será um dataframe referente aos respectivos devices 
for port in ports: 
  data = data.loc[ ~(data["Message Identifier"] == "1,2")]
  array_publish = data.loc[data["Source Port"] == port].dropna(subset=['Message Identifier'])
  array_publish_ack = data.loc[data["Destination Port"] == port].dropna(subset=['Message Identifier'])
  device = {"publish":array_publish, "publish_ack" : array_publish_ack}
  devices[port] = device

In [18]:
data = {}
df_concatenado = pd.DataFrame()
for port in ports:
    RTT = pd.DataFrame(columns=["Time_publish", "Time_ack"])
    for index, row in devices[port]["publish"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Size_Payload"] = 246 

        RTT.loc[id, "Time_publish"] = time_publish 

    for index, row in devices[port]["publish_ack"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_ack"] = time_publish
    RTT.loc[id, "Time_ack"] = time_publish
    RTT["RTT"]  =(RTT["Time_ack"] - RTT["Time_publish"] )* 1000
    data[port] = RTT
    df_concatenado = pd.concat([data[port], df_concatenado])
df_concatenado["RTT"].std()
media = df_concatenado["RTT"].mean()
e = 0.05 * media
n = (1.96 * (df_concatenado["RTT"].std()/e))**2
n


16.835468205094074

In [19]:

media = df_concatenado["RTT"].mean()
e = 0.05 * media
n = (1.96 * (df_concatenado["RTT"].std()/e))**2
n

16.835468205094074

# CÁLCULO DO TAMANHO DA AMOSTRA
***
## Variáveis quantitativas e população infinita
***
## Nívéis de confiança e significância 
O **nível de confiança** ($1 - \alpha$) representa a probabilidade de acerto da estimativa. De forma complementar o **nível de significância** ($\alpha$) expressa a probabilidade de erro da estimativa.

O **nível de confiança** representa o grau de confiabilidade do resultado da estimativa estar dentro de determinado intervalo. Quando fixamos em uma pesquisa um **nível de confiança** de 95%, por exemplo, estamos assumindo que existe uma probabilidade de 95% dos resultados da pesquisa representarem bem a realidade, ou seja, estarem corretos.

O **nível de confiança** de uma estimativa pode ser obtido a partir da área sob a curva normal como ilustrado na figura abaixo.


![alt text](https://caelum-online-public.s3.amazonaws.com/1178-estatistica-parte2/01/img007.png)

Em nosso trabalho iremos utilizar um nivel de confiança de 95%, logo abaixo iremos calcular o Z.

Esta variável $Z$ representa o afastamento em desvios padrões de um valor da variável original em relação à média.

In [20]:
#variavel nível de significancia
nivel_de_significancia = 0.05

In [21]:
#Calculando nivel de confiança
nivel_de_confiança = 1 - nivel_de_significancia
nivel_de_confiança

0.95

In [22]:
#Calculando o z com scipy
z = scipy.stats.norm.ppf(0.975)
z

1.959963984540054

Com nosso z calculado podemos seguir adiante

### $$z = 1.959963984540054 $$

Abaixo segue algumas formulas que iremos utilizar para calcular o numero de amostra

## $$n = \left(z\frac{s}{e}\right)^2$$





Onde:

$z$ = variável normal padronizada

$s$ = desvio padrão amostral

$e$ = erro inferencial


### Calculando a desvio padrão $s$

In [23]:
# Calculando o desvio padrão do nosso único dispositivo
desvio_padrao_amostral = df_concatenado["RTT"].std()
desvio_padrao_amostral

3.357909471699747

In [24]:
#Calculando o erro da nossa amostra equivalente a 5%

media = df_concatenado["RTT"].mean()
e = 0.05 * media
e

1.6040298848485879

### Calculando o tamanho da nossa amostra

# Substituindo todos os valores na formula para calcular o numero de amostra.

In [25]:
n = (z * (desvio_padrao_amostral/e))**2
n

16.83484949942058